### Import data

In [12]:
import urllib.request 

url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")

file_path = "the-verdict.txt"

urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x108ffdac0>)

### Read data

In [13]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("The total number of characters in the text is: ", len(raw_text))
print("The first 100 characters in the text are:")
print(raw_text[:99])
    


The total number of characters in the text is:  20479
The first 100 characters in the text are:
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Preprocess text (tokenization)


In [14]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text) # Split text into tokens

preprocessed = [item.strip() for item in preprocessed if item.strip()] # Remove empty strings
print(len(preprocessed)) # Print the number of tokens
print(preprocessed[:30]) # Print the first 10 tokens

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Convert tokens into token IDs



In [15]:
all_words = set(preprocessed)
vocab_size = len(all_words)
print("The total number of unique tokens in the raw text is: ", vocab_size)

The total number of unique tokens in the raw text is:  1130


In [18]:
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50: # Print only the first 50 tokens
        break 

('question', 0)
('appearance', 1)
('borne', 2)
('anything', 3)
('travelled', 4)
('disguised', 5)
('buying', 6)
('Now', 7)
('upstairs', 8)
('straight', 9)
('up-stream', 10)
('Hang', 11)
('look', 12)
('hot-house', 13)
('half-mechanically', 14)
('just', 15)
('poised', 16)
('about', 17)
('care', 18)
('our', 19)
('object', 20)
('cards', 21)
('failed', 22)
('hardly', 23)
('dining-room', 24)
('self-confident', 25)
('vista', 26)
('chap', 27)
('.', 28)
('Professional', 29)
('slight', 30)
('grew', 31)
('embarrassed', 32)
('quietly', 33)
('shall', 34)
('fancy', 35)
('deploring', 36)
('dead', 37)
('modest', 38)
('domestic', 39)
('insensible', 40)
('marble', 41)
('t', 42)
('straddling', 43)
('and', 44)
('had', 45)
('excuse', 46)
('equanimity', 47)
('doesn', 48)
('garlanded', 49)
('showed', 50)


### Simple text tokenizer

In [29]:
"""

This is a simple text tokenizer that takes a string and converts it into a list of token IDs.
It also has a method to decode token IDs back into text.
The tokenizer assumes that the input text is a sequence of words and spaces, 
and that the words are separated by spaces.
Punctuation marks are treated as separate tokens.
"""

class SimpleTextTokenizerV1:
    def __init__(self, vocab): # Constructor
        self.str_to_int = vocab # Map strings to integers
        self.int_to_str = {i:s for s, i in vocab.items()} # Map integers to strings

    def encode(self, text): # Encode text into token IDs
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # Split text into tokens
        preprocessed = [item.strip() for item in preprocessed if item.strip()] # Remove empty strings
        ids = [self.str_to_int[s] for s in preprocessed] # Map strings to integers
        return ids # Return the token IDs
    
    def decode(self, ids): # Decode token IDs into text
        text = " ".join([self.int_to_str[i] for i in ids]) # Join tokens with spaces

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # Join tokens with spaces
        return text

In [27]:
tokenizer = SimpleTextTokenizerV1(vocab)
text = """"It's the last he painted, you know,"
        Mrs.Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)


[82, 678, 504, 710, 870, 799, 948, 438, 210, 730, 387, 210, 82, 792, 28, 351, 565, 301, 1060, 862, 28]


In [30]:
print(tokenizer.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [33]:
# Try and encode text that is not in the vocabulary. Hello is not in the vocabulary.
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

KeyError: 'Hello'

### Add special context tokens

In [35]:
all_tokens = sorted(list(set(preprocessed))) # Sort the tokens
all_tokens.extend(["<|endoftext|>", "<|unk|>"]) # Add special end of text and unknow token
vocab = {token: integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [36]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)



('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


### Update tokenizer class

In [57]:
class SimpleTextTokenizerV2:
    def __init__(self, vocab): # Constructor
        self.str_to_int = vocab # Map strings to integers
        self.int_to_str = {i:s for s, i in vocab.items()} # Map integers to strings

    def encode(self, text): # Encode text into token IDs
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # Split text into tokens
        preprocessed = [item.strip() for item in preprocessed if item.strip()] # Remove empty strings 
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed] # Map strings to integers
        return ids # Return the token IDs
    
    def decode(self, ids): # Decode token IDs into text
        text = " ".join([self.int_to_str[i] for i in ids]) # Join tokens with spaces

        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) # Join tokens with spaces
        return text

In [58]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace." 
text = " <|endoftext|> ".join((text1, text2))
print(text)


Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [59]:
tokenizer = SimpleTextTokenizerV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [60]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.
